In [3]:
#極簡化的模型(不考慮各種glitch)

# Simplified "Mario-like" 1D runner MDP solved via dynamic programming (value iteration).
# --- ENVIRONMENT DESCRIPTION ---
# States: positions 0..9
# Actions: "run" (go forward), "jump" (go forward but avoids hazard)
# Hazards: pit at tile 3, enemy at tile 5 (run into them => death)
# Rewards: -1 every step, +100 reach goal, -100 death
# Goal: tile 9

from typing import Dict, Tuple
import pandas as pd

tiles = list(range(10))
start = 0
goal = 9
pits = {3}
enemies = {5}
actions = ["run", "jump"]

states = tiles + ["DEAD"]
terminal_states = {goal, "DEAD"}

def next_state_and_reward(s: int, a: str) -> Tuple[object, float]:
    if s in terminal_states:
        return s, 0.0
    target = s + 1
    if a == "run":
        if target in pits or target in enemies:
            return "DEAD", -100.0
        return (goal, 100.0) if target == goal else (target, -1.0)
    if a == "jump":
        return (goal, 100.0) if target == goal else (target, -1.0)

P = {s: {a: next_state_and_reward(s, a) for a in actions} for s in tiles}
P["DEAD"] = {a: ("DEAD", 0.0) for a in actions}
P[goal] = {a: (goal, 0.0) for a in actions}

gamma = 1.0
V = {s: 0.0 for s in states}

def value_iteration(P, V, actions, theta=1e-6, max_iters=1000):
    for _ in range(max_iters):
        delta = 0.0
        V_new = V.copy()
        for s in tiles:
            if s in terminal_states: 
                continue
            Qs = [P[s][a][1] + gamma * V[P[s][a][0]] for a in actions]
            V_new[s] = max(Qs)
            delta = max(delta, abs(V_new[s] - V[s]))
        V = V_new
        if delta < theta:
            break
    return V

V = value_iteration(P, V, actions)

policy = {}
for s in tiles:
    if s in terminal_states:
        policy[s] = None
        continue
    best_a = max(actions, key=lambda a: P[s][a][1] + V[P[s][a][0]])
    policy[s] = best_a

trajectory = []
s = start
while s not in terminal_states:
    a = policy[s]
    s_next, r = P[s][a]
    trajectory.append((s, a, s_next, r))
    s = s_next

df = pd.DataFrame(trajectory, columns=["state","action","next_state","reward"])
print(df)
print("\nTotal reward:", df["reward"].sum())
print("Steps:", len(df))
print("\nOptimal policy per tile:", policy)


   state action  next_state  reward
0      0    run           1    -1.0
1      1    run           2    -1.0
2      2   jump           3    -1.0
3      3    run           4    -1.0
4      4   jump           5    -1.0
5      5    run           6    -1.0
6      6    run           7    -1.0
7      7    run           8    -1.0
8      8    run           9   100.0

Total reward: 92.0
Steps: 9

Optimal policy per tile: {0: 'run', 1: 'run', 2: 'jump', 3: 'run', 4: 'jump', 5: 'run', 6: 'run', 7: 'run', 8: 'run', 9: None}


In [6]:
#以Bullet Bill為例，說明發現glitch的過程
# bullet_bill_demo.py
# Simplified "Bullet Bill" model (Design B) + lightweight Go-Explore style search
# - BulletBillEnv: deterministic env with RAM and save/load
# - RandomExplorer: performs archive-based exploration to try to find warp_byte != expected (bullet bill)

from dataclasses import dataclass, field
import random, hashlib, json
from typing import List, Any

@dataclass
class BulletBillEnv:
    level_length: int = 30
    goal_pos: int = 29
    player_start: int = 0
    warp_addr: int = 0
    ram_size: int = 16
    rng_seed: int = 12345
    
    pos: int = field(init=False, default=0)
    subpixel: float = field(init=False, default=0.0)
    frame: int = field(init=False, default=0)
    ram: List[int] = field(init=False, default_factory=list)
    rng_state: Any = field(init=False, default=None)
    action_history: List[int] = field(init=False, default_factory=list)
    done: bool = field(init=False, default=False)
    info: dict = field(init=False, default_factory=dict)
    
    def reset(self, seed: int = None):
        if seed is None:
            seed = self.rng_seed
        self.rng_state = random.Random(seed)
        self.pos = self.player_start
        self.subpixel = 0.0
        self.frame = 0
        self.ram = [0] * self.ram_size
        self.ram[self.warp_addr] = 1
        self.action_history = []
        self.done = False
        self.info = {}
        self.special_positions = self._precompute_special_positions()
        return self._get_obs()
    
    def _precompute_special_positions(self):
        s = {}
        r = self.rng_state
        for f in range(0, 200):
            if f % 7 == 0:
                p = (5 + (f * 3 + r.randint(0,3)) % (self.level_length - 10))
                s[f] = {p}
            else:
                s[f] = set()
        return s
    
    def _get_obs(self):
        return {
            "pos": self.pos,
            "subpixel": round(self.subpixel,3),
            "ram": tuple(self.ram),
            "frame": self.frame,
            "special_positions": tuple(sorted(self.special_positions.get(self.frame, set())))
        }
    
    def save_state(self):
        return {
            "pos": self.pos,
            "subpixel": self.subpixel,
            "frame": self.frame,
            "ram": list(self.ram),
            "rng_state": self.rng_state.getstate(),
            "action_history": list(self.action_history)
        }
    
    def load_state(self, state):
        self.pos = state["pos"]
        self.subpixel = state["subpixel"]
        self.frame = state["frame"]
        self.ram = list(state["ram"])
        r = random.Random()
        r.setstate(state["rng_state"])
        self.rng_state = r
        self.action_history = list(state["action_history"])
        self.done = False
        self.info = {}
        self.special_positions = self._precompute_special_positions()
        return self._get_obs()
    
    def step(self, action: int):
        if self.done:
            return self._get_obs(), 0.0, True, self.info
        self.frame += 1
        self.action_history.append(action)
        
        # motion model: action -> subpixel advance
        if action == 0:
            adv = 0.0
        elif action == 1:
            adv = 0.25
        elif action == 2:
            adv = 0.25
        elif action == 3:
            adv = 0.0
        else:
            adv = 0.0
        self.subpixel += adv
        while self.subpixel >= 1.0:
            self.subpixel -= 1.0
            self.pos += 1
        if self.pos >= self.level_length:
            self.pos = self.level_length - 1
        
        collided = False
        special_here = self.special_positions.get(self.frame, set())
        if self.pos in special_here:
            # glitch trigger condition: exact subpixel == 0.5 and action == 2
            if abs(self.subpixel - 0.5) < 1e-9 and action == 2:
                self.ram[self.warp_addr] = 8  # Bullet Bill trigger (corrupt warp byte)
                collided = True
            else:
                # normal collision effect: knockback + minor RAM side-effect
                self.pos = max(0, self.pos - 1)
                self.ram[5] = (self.ram[5] + 1) % 256
        
        reward = -1.0
        done = False
        if self.pos >= self.goal_pos:
            reward += 200.0
            done = True
            self.done = True
        
        info = {"warp_byte": self.ram[self.warp_addr], "collided_special": collided}
        self.info = info
        return self._get_obs(), reward, done, info

def state_hash(state_obs):
    ram = state_obs["ram"]
    key = (ram[0], ram[1], ram[2], state_obs["pos"], round(state_obs["subpixel"],3))
    return hashlib.md5(json.dumps(key).encode()).hexdigest()

class RandomExplorer:
    def __init__(self, env: BulletBillEnv, max_iters=500, explore_steps=30):
        self.env = env
        self.archive = {}
        self.max_iters = max_iters
        self.explore_steps = explore_steps
        self.found_traces = []
    def initialize(self, seed=123):
        obs = self.env.reset(seed)
        s = self.env.save_state()
        h = state_hash(obs)
        self.archive[h] = {"save": s, "obs": obs, "trace": []}
    def pick_entry(self):
        return random.choice(list(self.archive.keys()))
    def explore(self):
        for it in range(self.max_iters):
            h = self.pick_entry()
            entry = self.archive[h]
            env_state = entry["save"]
            obs = self.env.load_state(env_state)
            local_trace = []
            found = False
            for step in range(self.explore_steps):
                if random.random() < 0.12:
                    action = 2
                else:
                    action = random.choice([0,1,3])
                obs, rew, done, info = self.env.step(action)
                local_trace.append(action)
                h2 = state_hash(obs)
                if h2 not in self.archive:
                    self.archive[h2] = {"save": self.env.save_state(), "obs": obs, "trace": entry["trace"] + local_trace.copy()}
                if info.get("warp_byte", None) != 1:
                    found = True
                    self.found_traces.append({"origin": h, "steps": step+1, "trace": entry["trace"] + local_trace.copy(), "obs": obs, "info": info})
                    h_new = state_hash(obs)
                    self.archive[h_new] = {"save": self.env.save_state(), "obs": obs, "trace": entry["trace"] + local_trace.copy()}
                    break
                if done:
                    break
            if found:
                print(f"[Explorer] Found bullet bill at iter {it}, steps={step+1}, total_trace_len={len(entry['trace'])+len(local_trace)}")
                return True
        return False

if __name__ == "__main__":
    # Run a short demo with limited budget
    env = BulletBillEnv(rng_seed=2025)
    expl = RandomExplorer(env, max_iters=100000, explore_steps=2000)
    expl.initialize(seed=2025)
    found = expl.explore()

    print("\nArchive size:", len(expl.archive))
    print("Found bullet bill?", found)
    if found:
        rec = expl.found_traces[-1]
        act_map = {0:"noop",1:"right",2:"right+jump",3:"jump"}
        print("Readable trace:", [act_map[a] for a in rec["trace"]])
        print("Final obs:", rec["obs"])
        print("Info:", rec["info"])
    else:
        print("No bullet bill found in budget. Increase max_iters/explore_steps or bias action 2 more.")



[Explorer] Found bullet bill at iter 19071, steps=145, total_trace_len=182

Archive size: 121
Found bullet bill? True
Readable trace: ['noop', 'noop', 'right', 'right', 'right', 'jump', 'right+jump', 'noop', 'jump', 'jump', 'jump', 'jump', 'jump', 'right', 'noop', 'right+jump', 'jump', 'noop', 'noop', 'noop', 'noop', 'right+jump', 'jump', 'jump', 'noop', 'right', 'right', 'noop', 'noop', 'jump', 'jump', 'noop', 'jump', 'jump', 'jump', 'noop', 'right', 'jump', 'right', 'noop', 'jump', 'jump', 'noop', 'right+jump', 'noop', 'right', 'right+jump', 'noop', 'right', 'noop', 'jump', 'right', 'jump', 'jump', 'jump', 'jump', 'jump', 'right', 'noop', 'noop', 'noop', 'noop', 'right', 'noop', 'noop', 'right', 'jump', 'right+jump', 'right', 'noop', 'noop', 'right+jump', 'noop', 'noop', 'right', 'jump', 'right', 'right', 'right+jump', 'jump', 'right+jump', 'noop', 'jump', 'noop', 'jump', 'jump', 'jump', 'right+jump', 'noop', 'noop', 'noop', 'jump', 'jump', 'right', 'right', 'noop', 'right', 'noop', 